In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)
filename_grokKey = "..\\GroqAPIKey.txt"
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)

### Tools

#### Creating own custom tool

In [11]:
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_google_genai import ChatGoogleGenerativeAI

In [7]:
@tool
def name_of_tool(input:str)-> str:
    """Tool Description"""
    return "Result"

In [8]:
print(name_of_tool.name)
print(name_of_tool.description)
print(name_of_tool.args)

name_of_tool
Tool Description
{'input': {'title': 'Input', 'type': 'string'}}


#### Defining, Binding and Calling the tool

In [10]:
@tool
def convert_usd_to_coins(input:float) -> float:
    """Use this tool to convert the USD to digital coins"""
    return 1.3*(float(input))

In [12]:
llm= ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')

In [13]:
llm_bind_tools = llm.bind_tools([convert_usd_to_coins])

In [14]:
result = llm_bind_tools.invoke("How many digital coins can I get for USD10")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert_usd_to_coins', 'arguments': '{"input": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-001', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f06f3a91-9b51-413a-92d1-5bc4892e77e4-0', tool_calls=[{'name': 'convert_usd_to_coins', 'args': {'input': 10.0}, 'id': '46799ea4-3be9-4262-a7ba-d677f26c4130', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 20, 'total_tokens': 78, 'input_token_details': {'cache_read': 0}})

In [15]:
result.tool_calls

[{'name': 'convert_usd_to_coins',
  'args': {'input': 10.0},
  'id': '46799ea4-3be9-4262-a7ba-d677f26c4130',
  'type': 'tool_call'}]

In [16]:
tool_mapping = {
    'convert_usd_to_coins': convert_usd_to_coins
}
tool_mapping

{'convert_usd_to_coins': StructuredTool(name='convert_usd_to_coins', description='Use this tool to convert the USD to digital coins', args_schema=<class 'langchain_core.utils.pydantic.convert_usd_to_coins'>, func=<function convert_usd_to_coins at 0x00000255F354D7E0>)}

In [17]:
tool = tool_mapping[result.tool_calls[0]["name"]]

In [18]:
tool_output = tool.invoke(result.tool_calls[0]["args"])

In [19]:
tool_output

13.0

##### Demonstration of the code to run the multiple tool calls in LLM response, we can Loop through the list of tools in tool call

In [ ]:
tool_mapping = {
    'convert_usd_to_coins': convert_usd_to_coins,
    'tools_name':tool2
}
tool_mapping

In [ ]:
for tool_call in result.tool_calls:
    tool = tool_mapping[tool_call["name"]]
    tool_output=tool.invoke(tool_call["args"])